# label data
- determine biosamples related to lung and lung cancer
- prepare ids.csv for nextflow pipeline downloading FASTQ

In [1]:
import json
import os
import numpy as np
import pandas as pd

from utils import Utils
from label_data import LabelData
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from parse_sra import ParseSra
from slicer import Slicer

%reload_ext autoreload
%autoreload 2


## prepare

In [2]:
# the direcotry stores soft files
data_dir = '/home/yuan/data'

# exported labeling data
label_dir = '../data/labels'
if not os.path.isdir(label_dir):
    os.mkdir(label_dir)
#
fastq_dir = '/home/yuan/rawdata/SRX'

In [9]:
# test parse, and enrich
geo = "GSE135893"
data_dir = '/home/yuan/data'
parser = ParseSoft(data_dir)
soft_path = parser.soft_local_path(geo)
print(soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)
# for item in data['SAMPLE']:
#     print(item['SAMPLE'])
# print(json.dumps(data['SAMPLE'], indent=4))
enriched = EnrichSoft(data)()

# save
LabelData('../data').save(enriched)

/home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE135nnn/GSE135893/soft/GSE135893_family.soft.gz


'/home/yuan/bio/scrnaseq_reference/data/labels/GSE135nnn/GSE135893.json'

## select GEO
keywords: human, lung, scRNA-seq

In [11]:
exclude_geo = ['GSE127462']
# 
total_geo = ['GSE286399', 'GSE280041', 'GSE144357', 'GSE121309', 'GSE183590', 'GSE148729']

In [12]:
# load pairwising data
pmid_geo = Utils.from_json('../results/pmid_geo.json')

cellmarker = pd.ExcelFile('../results/Cell_marker_Seq.xlsx')
cm = cellmarker.parse('seq')

# get PMID
lung_cm = cm[(cm['species']=='Human') & cm['tissue_type'].str.contains('Lung')]
pmid_list= np.unique(lung_cm['PMID'])
print(len(pmid_list))

n = 0
for pmid in pmid_list:
    pmid_keys = Slicer.PMID(pmid)
    geo_pool = Utils.key_get(pmid_geo, pmid_keys)
    for geo in geo_pool:
        if geo not in total_geo + exclude_geo:
            total_geo.append(geo)
        n += 1
print(f"{n} GEO datasets are detected.")

82
34 GEO datasets are detected.


In [13]:
# include all PMID human related
human_cm = cm[cm['species']=='Human']
is_lung = human_cm.apply(lambda x: x.str.lower().str.contains('lung').any(), axis=1)
human_cm = human_cm[is_lung]
human_pmid_list= np.unique(human_cm['PMID'])
print(len(human_pmid_list))

n = 0
for pmid in human_pmid_list:
    pmid_keys = Slicer.PMID(pmid)
    geo_pool = Utils.key_get(pmid_geo, pmid_keys)
    for geo in geo_pool:
        if geo not in total_geo + exclude_geo:
            total_geo.append(geo)
        n += 1
print(f"{n} GEO datasets are detected.")

110
53 GEO datasets are detected.


In [15]:
# finally create GEO meta data
m = n = 0
for geo in total_geo:
    print(geo, end=', ')
    soft_path = ParseSoft(data_dir).soft_local_path(geo)
    softer = SearchSoft(soft_path)
    data = softer.filter_data(softer.parse_rows)
    enriched = EnrichSoft(data)()
    if '9606' in enriched['taxid']:
        LabelData('../data').save(enriched)
        print(geo, end=', ')
        n += 1
    else:
        m += 1
print(f'\n{n} GEO data are created. {m} are skipped')

GSE286399, GSE286399, GSE280041, GSE280041, GSE144357, GSE144357, GSE121309, GSE121309, GSE183590, GSE183590, GSE148729, GSE148729, GSE112274, GSE112274, GSE121611, GSE122960, GSE122960, GSE128033, GSE128033, GSE128169, GSE128169, GSE133747, GSE133747, GSE137811, GSE137811, GSE137805, GSE137805, GSE137799, GSE137799, GSE124885, GSE124885, GSE132771, GSE132771, GSE147066, GSE147066, GSE145926, GSE145926, GSE140819, GSE140819, GSE135851, GSE135851, GSE135893, GSE135893, GSE136831, GSE136831, GSE217722, GSE217722, GSE162936, GSE162936, GSE162499, GSE162499, GSE162500, GSE162500, GSE162498, GSE162498, GSE158055, GSE158055, GSE166059, GSE166059, GSE166034, GSE166034, GSE166036, GSE166036, GSE166033, GSE166033, GSE166035, GSE166035, GSE166037, GSE166037, GSE168710, GSE168710, GSE148071, GSE148071, GSE164829, GSE164829, GSE156311, GSE168299, GSE127472, GSE127472, GSE127813, GSE127813, GSE127471, GSE127471, GSE130148, GSE130148, GSE125188, GSE125188, GSE158127, GSE158127, GSE161089, GSE161089,

## parse SRR given BioSample and SRX
- parse SRR accession give biosample or SRX accesssion from local file
- parse URLS of *.fastq.gz given a SRR accession from FTP (deprecated)
- parse URLS of *.fastq.gz given a SRR accession from local json

In [49]:

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# step 1: Given BioSample, parse SRR accessions if they exist
from parse_sra import ParseSra
from utils import Utils
from label_data import LabelData

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')

label_dir = '../data/labels'
for data in Utils.json_iter(label_dir):
    data = ParseSra.parse_srr(data, samn_srr)
    LabelData('../data').save(data)

In [37]:
# step 2: retrieve URLs of SRR*.gz from 'ftp.sra.ebi.ac.uk'
# only retrieve current GEO datasets
import os
import time
from utils import Utils
from retrieve_url import RetrieveUrl
from slicer import Slicer

def get_srr(label_dir):
    for data in Utils.json_iter(label_dir):
        geo = data['GEO']
        print(geo, end=',')
        samples = data.get('samples', {})
        for sample_id in samples:
            SRR = samples[sample_id].get('SRR', {})
            yield geo, set(SRR)

def retrieve_fastq_urls(srr_iter, outdir):
    file_name = 'srr_fastq_urls_simple.json'
    urls_json = os.path.join(outdir, file_name)
    urls = Utils.from_json(urls_json)
    print('number of GEO datasets = ', len(urls))
    n = m = p = 0
    for geo, srr_pool in srr_iter:
        n += len(srr_pool)
        if geo not in urls:
            urls[geo] = {}
        pool = srr_pool.difference(set(urls[geo]))
        for srr_acc in pool:
            gz = RetrieveUrl.ftp_sra_ebi(srr_acc)
            if gz:
                urls[geo][srr_acc] = gz
                m += 1
            else:
                p += 1
            if m > 0 and m % 20 == 0:
                Utils.to_json(urls, outdir, file_name)
                time.sleep(2)
                print(f"{n}:{m},{p}", end='\t')
    if m > 0:
        Utils.to_json(urls, outdir, file_name)
    print(f"Among {n} SRR accessions, {m} are newly added. The URLs of {p} are unknown")
    return None
# 
outdir = '../results'
label_dir = '../data/labels'
srr_iter = get_srr(label_dir)
retrieve_fastq_urls(srr_iter, outdir)

number of GEO datasets =  51
GSE135893,GSE130148,GSE286399,GSE148071,GSE148729,GSE127813,GSE127472,GSE127471,GSE122960,GSE190510,GSE121309,GSE196303,GSE132771,GSE180063,GSE180908,GSE144357,GSE155249,GSE155515,GSE147066,GSE161089,GSE137799,GSE137811,GSE137805,GSE164829,GSE158055,GSE158127,GSE124885,GSE136831,GSE162499,GSE162936,GSE162498,GSE162500,Among 8212 SRR accessions, 0 are newly added. The URLs of 66 are unknown


In [20]:
# step 3: merge srr_urls into data
# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
outdir = '../results'
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join(outdir, file_name)
urls = Utils.from_json(urls_json)

n = 1
headers = ['No.', 'GEO', 'SRR', 'available', 'unknown', 'updated']
width = [5, 15, 12,12, 10, 10]
headers = [h.ljust(w) for h,w in zip(headers, width)]
print(''.join(headers))
key = 'ftp.sra.ebi.ac.uk'
for data in Utils.json_iter(label_dir):
    srr = updated = available = unknown = 0
    geo = data['GEO']
    samples = data.get('samples', {})
    for sample in samples.values():
        srr += 1
        for srr_acc in sample.get('SRR', {}):
            if key not in sample['SRR'][srr_acc]:
                if srr_acc in urls[geo]:
                    sample['SRR'][srr_acc][key] = urls[geo][srr_acc]
                    updated += 1
            if key in sample['SRR'][srr_acc]:
                available += 1
            else:
                unknown += 1
    if updated > 0:
        LabelData('../data').save(data)
    info = [n, geo, srr, available, unknown, updated,]
    info = [str(h).ljust(w) for h,w in zip(info, width)]
    print(''.join(info))

No.  GEO            SRR         available   unknown   updated   
1    GSE133747      20          8           0         0         
1    GSE128033      18          0           0         0         
1    GSE128169      16          0           0         0         
1    GSE154826      166         250         0         0         
1    GSE112274      1090        2725        0         0         
1    GSE280041      104         156         0         0         
1    GSE168710      4           0           0         0         
1    GSE140819      40          0           0         0         
1    GSE166059      90          96          0         0         
1    GSE166034      14          14          0         0         
1    GSE166033      10          10          0         0         
1    GSE166037      2           8           0         0         
1    GSE166035      15          15          0         0         
1    GSE166036      49          49          0         0         
1    GSE145926      21   

In [66]:
# Note: old step2 deprecated
# step 2_old: data are determined previously. scan entire FTP
# retrieve URLs as much as possible. time-consuming
from utils import Utils
srr_fastq = Utils.from_json('../results/srr_fastq_urls_1.json')
srr_fastq = srr_fastq['vol1']['fastq']
print(len(srr_fastq))

992


In [ ]:
# Note: old step2 deprecated
# step 3_old: retrieve FTP URLs of *gz files given their SRR accessions
# Note: run the process multiple times to make sure all SRRs are accessed
import time
from parse_sra import ParseSra
from utils import Utils
from label_data import LabelData

n = 0
headers = ['No.', 'GEO', 'SRR', 'ready', 'unknown', 'local', 'FTP', 'unavailable']
width = [5, 15, 12,12, 10, 10, 10, 10]
headers = [h.ljust(w) for h,w in zip(headers, width)]
print(''.join(headers))
for data in Utils.json_iter(label_dir):
    unavailable = 1
    while unavailable:
        if unavailable > 1:
            print(f'{unavailable}...')
            time.sleep(2)            
        data, info, unavailable = ParseSra.parse_ftp_fastq(data, srr_fastq)
        LabelData(label_dir).save(data)
    n += 1
    info = [str(n),] + info
    info = [h.ljust(w) for h,w in zip(info, width)]
    print(''.join(info))

In [2]:
# step 4: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from utils import Utils
from parse_sra import ParseSra
from label_data import LabelData

fastq_dir = '/home/yuan/rawdata/SRR'
for data in Utils.json_iter('../data/labels'):
    data, unparsing = ParseSra.parse_local_fastq(data, fastq_dir)
    if unparsing:
        print(data['GEO'], len(unparsing), unparsing[:6])
    LabelData('../data').save(data)

GSE166059 1 ['SRR13615084']
GSE135851 8 ['SRR9970044', 'SRR9970045', 'SRR9970046', 'SRR9970047', 'SRR9970048', 'SRR9970049']
5000, GSE144357 16 ['SRR10974649', 'SRR10974717', 'SRR10974718', 'SRR10974752', 'SRR10976738', 'SRR10976779']
GSE162499 2 ['SRR13181475', 'SRR13181478']
GSE162498 17 ['SRR13181483', 'SRR13181484', 'SRR13181485', 'SRR13181486', 'SRR13181487', 'SRR13181488']
GSE162500 29 ['SRR13181483', 'SRR13181484', 'SRR13181485', 'SRR13181486', 'SRR13181487', 'SRR13181488']


## select biosamples and 

In [65]:
# prepare SRR id list for download
num_geo = num_sample = 0
cl = LabelData(label_dir)
with open('../results/srr_ids.txt', 'w') as f:
    for data in cl.json_iter():
        if data['taxid'] == '9606':
            num_geo += 1
            for sample_id, sample in data['samples'].items():
                if sample.get('scrnaseq') and sample['SRR']:
                    #print(data['GEO'], sample_id)
                    num_sample += 1
                    for srr_acc, local_path in sample['SRR'].items():
                        if not local_path:
                            # print(f"fastq-dump {srr_acc} -O /home/yuan/results/fastq --gzip")
                            f.write(srr_acc + '\n')
print(num_geo, num_sample)

52 400


In [87]:
def lung_cancer_cell_line(df):
    res = []
    for name in ['H1299', 'Calu3', 'A549']:
        df1 = df[df['cell_line']==name]
        print(name, len(df1))
        for sample_id in list(df1.index):
            for srr_acc, fq in sample_srr[sample_id].items():
                if fq:
                    label, fq1, fq2 = f"{name}_{srr_acc}", fq.get('R1'), fq.get('R2')
                    res.append((label, fq1, fq2))
    res = pd.DataFrame(res, columns=['sample', 'fastq_1', 'fastq_2'])
    return res
df1 = lung_cancer_cell_line(df)
df1.to_csv('../data/samplesheet_cellline_lungcancer.csv', index=False)
df1.head()

H1299 40
Calu3 90
A549 1


,sample,fastq_1,fastq_2
0,H1299_SRR11549938,/home/yuan/data/SRA/SRR115/038/SRR11549938.fas...,None
1,H1299_SRR11549939,/home/yuan/data/SRA/SRR115/039/SRR11549939.fas...,None
2,H1299_SRR11549940,/home/yuan/data/SRA/SRR115/040/SRR11549940.fas...,None
3,H1299_SRR11549941,/home/yuan/data/SRA/SRR115/041/SRR11549941.fas...,None
4,H1299_SRR11549942,/home/yuan/data/SRA/SRR115/042/SRR11549942.fas...,None


### lung cell line

 

### colon cancer, key=cell_line
- HCT 116 is an adherent cell line isolated from the colon of a patient with colon cancer. It has a mutation in codon 13 of the ras-proto-oncogene. This cell line is near-diploid and has a relatively stable genetic profile, making it a valuable in vitro model. This line can be utilized in cancer research and gastrointestinal (GI) research.
- Caco-2 [Caco2] is an adherent cell line isolated from colon tissue derived from a patient with colorectal adenocarcinoma. 

### normal cell line
- cell_type = "airway basal stem cells"
- cell_type = "CD45+", group="healthy control"